# Using Multiple Pilots

RADICAL-Pilot supports managing multiple pilots during a single run, while workload (bag of computing tasks) will be distributed among all available pilots. [rp.TaskManager](../apidoc.rst#radical.pilot.TaskManager) dispatches tasks to pilots according to a scheduling algorithm ([rp.SCHEDULER_ROUND_ROBIN](../apidoc.rst#radical.pilot.TaskManager.scheduler), [rp.SCHEDULER_BACKFILLING](../apidoc.rst#radical.pilot.TaskManager.scheduler)), which is provided during its initialization.

<div class="alert alert-info">

__Note:__ RADICAL-Pilot enacts a second scheduling step within each launched pilot: once the Agent takes ownership of tasks, which the [rp.TaskManager](../apidoc.rst#radical.pilot.TaskManager) scheduler assigned to it, the Agent Scheduler will place the tasks on the set of resources (CPU cores, GPUs) that the Agent is managing. The Agent Scheduler can be configured via resource configuration files (see the tutorial [RADICAL-Pilot Configuration System](configuration.ipynb))

</div>

* Round-Robin Scheduler, RR (`rp.SCHEDULER_ROUND_ROBIN`, __default__) fairly distributes arriving tasks over the set of known pilots, independent of task state, expected workload, pilot state or pilot lifetime. As such, RR is a fairly simplistic, but also a very fast scheduler, which does not impose any additional communication round trips between [rp.TaskManager](../apidoc.rst#radical.pilot.TaskManager) and pilot agents.

* Backfilling Scheduler, BF (`rp.SCHEDULER_BACKFILLING`) provides a better load balancing, but at the cost of additional communication round trips. It depends on the actual application workload if that load balancing is beneficial or not. It is most beneficial for the large number of pilots and for relatively long-running tasks (e.g., the task runtime is significantly longer than the communication round trip time between [rp.TaskManager](../apidoc.rst#radical.pilot.TaskManager) and pilot agents).

   * It is __not recommended__ to use BF for: (i) a single pilot, and/or (ii) many short-running tasks.
   * BF will only dispatch tasks to pilot agents once the pilot agent is in the `rp.PMGR_ACTIVE` state. The tasks will thus get executed even if one of the pilots never reaches that state - the load will be distributed between pilots which become active (`rp.PMGR_ACTIVE`).
   * BF will only dispatch as many tasks to the Agent as they can be executed concurrently. No tasks will be waiting in the Agent Scheduler queue. BF will react on task termination events, and will then backfill (!) the Agent with any remaining tasks. The Agent will remain under-utilized during that communication.

## Examples

<div class="alert alert-info">

__Note:__ In our examples, we will not show a progression bar while waiting for some operation to complete, e.g., while waiting for a pilot to stop. That is because the progression bar offered by RP's reporter does not work within a notebook. You could use it when executing an RP application as a standalone Python script.

</div>


In [ ]:
%env RADICAL_REPORT=TRUE
%env RADICAL_REPORT_ANIME=FALSE

In [ ]:
import radical.pilot as rp
import radical.utils as ru

In [ ]:
session = rp.Session()
pmgr    = rp.PilotManager(session=session)
tmgr    = rp.TaskManager(session=session)  # -> scheduler=rp.SCHEDULER_ROUND_ROBIN

We run multiple pilots using a description for a "localhost" platform (`local.localhost`), which is provided by the built-in configuration file `resource_local.json` and mimics a configuration for CPU cores and GPUs.

<div class="alert alert-info">

__Note:__ The amount of resources for "localhost" is arbitrary, and it does not reflect the number of physical resources available.

</div>

<div class="alert alert-info">

__Note:__ Before running multiple pilots on a single HPC platform, ensure that its scheduling policy conforms the use case, since each pilot represents a batch job and HPC's scheduling policy might limit the number of jobs running simultaneously per user. For the remote submission see the tutorial [Using RADICAL-Pilot on HPC Platforms](submission.ipynb).

</div>

In [ ]:
pilots = []

# submission of a single pilot
pd0 = rp.PilotDescription({
    'resource': 'local.localhost',
    'cores'   : 4,
    'runtime' : 10
})
pilots.append(pmgr.submit_pilots(pd0))

# submission of multiple pilots at the same time
pd1 = rp.PilotDescription({
    'resource': 'local.localhost',
    'cores'   : 2,
    'runtime' : 7
})
pd2 = rp.PilotDescription({
    'resource': 'local.localhost',
    'cores'   : 3,
    'runtime' : 7
})
pilots.extend(pmgr.submit_pilots([pd1, pd2]))

After pilots are submitted, they should be added to [rp.TaskManager](../apidoc.rst#radical.pilot.TaskManager) for the task dispatching.

In [ ]:
tmgr.add_pilots(pilots)
tmgr.list_pilots()  # lists all pilots available for TaskManager

In [ ]:
N_TASKS = 4
tds     = []  # list of task descriptions

for _ in range(N_TASKS):

    td = rp.TaskDescription({
        'executable': '/bin/echo',
        'arguments' : ['pilot_id=$RP_PILOT_ID']
    })

    tds.append(td)

tasks = tmgr.submit_tasks(tds)
tmgr.wait_tasks()

Each task was assigned to a corresponding pilot according to a default scheduling algorithm `rp.SCHEDULER_ROUND_ROBIN`, but it is possible to assign a task to a particular pilot explicitly with the [rp.TaskDescription.pilot](../apidoc.rst#radical.pilot.TaskDescription.pilot) attribute. Thus, in the example below, we submit another pilot and assign a new task to it.

In [ ]:
pd3 = rp.PilotDescription({
    'resource': 'local.localhost',
    'cores'   : 2,
    'runtime' : 5
})

pilot = pmgr.submit_pilots(pd3)
tmgr.add_pilots(pilot)

td = rp.TaskDescription()
td.executable = '/bin/echo'
td.arguments  = ['task is assigned to $RP_PILOT_ID']
td.pilot      = pilot.uid

task = tmgr.submit_tasks(td)
tmgr.wait_tasks(uids=task.uid)

tasks.append(task)

If the task is assigned to a pilot, which wasn't added to [rp.TaskManager](../apidoc.rst#radical.pilot.TaskManager), then that pilot is unknown for the task dispatching, and the task is pending for a corresponding pilot to arrive. In the example below, we set a timeout for the [rp.TaskManager.wait_tasks()](../apidoc.rst#radical.pilot.TaskManager.wait_tasks) method, since by default, that method waits for tasks reaching their final state, but this task will have the state `rp.TMGR_SCHEDULING` until the session will be closed (since we will not submit an associated pilot).

In [ ]:
td = rp.TaskDescription()

td.executable = '/bin/echo'
td.arguments  = ['pilot_id=$RP_PILOT_ID']
td.pilot      = 'unknown_pilot_id'

task = tmgr.submit_tasks(td)
tmgr.wait_tasks(uids=task.uid, timeout=15)

As a final step, we go through all tasks outputs to see which pilot was assigned to each task.

In [ ]:
for task in tasks:
    stdout = task.stdout.strip()[:35]
    print('%s - output: %-30s (in task description: pilot="%s")' %
          (task.uid, stdout, task.description['pilot']))

In [ ]:
session.close()